Solve a Second Order Cone Programming (SOCP) in the form:
\begin{equation*}
\begin{aligned}
\underset{x}{\text{minimize}}
 \; c^{T} \cdot x \\
 \text{subject to}
\; A \cdot b = b \\
\; G \cdot x \leq k_{h}
\end{aligned}
\end{equation*}
The arguments $\mathrm{c}$, $\mathrm{h}$, and $\mathrm{b}$ are Numpy arrays (i.e. matrices with a single column). The arguments $\mathrm{G}$ and $\mathrm{A}$ are Scipy sparse matrices in compressed sparse column (CSC) format; if they are not of the proper format (e.g. in compressed sparse row (CSR) format),ECOS will attempt to convert them. 
The argument dims is a dictionary with three fields, `dims['l']`, `dims['q']` and `dims['e']`; 
-  `dims['l']` - scalar, dimension of positive orthant (LP-cone) `R_+`;
-  `dims['q']` - vector with dimensions of second order cones;
-  If the fields are omitted or empty, they default to 0. 

The argument kwargs can include the keywords:
- `feastol`, `abstol`, `reltol`, `reltol`, `abstol_innac`, `reltol_inacc`: tolerance values; 
- `max_iters`: maximum number of iterations; 
- `verbose` : Boolean; 
- arguments `A`, `b`, and kwargs are optional.

The returned object is a dictionary containing the fields:
- `solution['x']`: primal variables
- `solution['y']`: dual variables for equality constraints
- `solution['s']`: slacks for `Gx + s <= h, s \in K`
- `solution['z']`: dual variables for inequality constraints `s \in K`
- `solution['info']`. 

More info in this [link](https://github.com/embotech/ecos-python).

In [1]:
# import library
import ecos
import scipy.sparse as sparse
import numpy as np

## A SOCP example

In [3]:
# set dimensions and sparsity of A
n       = 1000
m       = 10

# linear term
c = np.concatenate((np.zeros(n), np.ones(n)), axis = 0)

# equality constraints
A      = np.random.randn(m, n)
Atilde = np.concatenate((A, np.zeros((m,n))) , axis = 1)

b = np.random.randn(m)
# linear inequality constraints
I = np.identity(n) 
row1 = np.concatenate((I, -I), axis = 1)               
row2 = np.concatenate((-I, -I), axis = 1)               
G    = np.concatenate((row1,row2), axis = 0)

h = np.zeros(2*n)
 
# cone dimensions (LP cone only)
dims = {} 
dims['l'] = 2*n #dims.l = 2*n;
dims['q'] = [] #dims.q = [];

G      = sparse.csr_matrix(G)
Atilde = sparse.csr_matrix(Atilde)

# call solver
solution = ecos.solve(c,G,h,dims,Atilde,b);

x = solution['x']       # primal variables
y = solution['y']       # dual variables for equality constraints
s = solution['s']       # slacks for Gx + s <= h, s \in K
z = solution['z']       # dual variables for inequality constraints s \in K
info = solution['info'] # exit_flag = 1 :KKT optimality conditions satisfied 

## Formulation of QP via ECOS

Attempts to solve the quadratic program (QP):
\begin{equation*}
\begin{aligned}
\; \underset{x}{\text{minimize}}
\; \frac{1}{2} x^{T} P x + q^{T} x \\
\; \text{subject to}
\;  l \leq A x \leq u \\
\end{aligned}
\end{equation*}

Assuming that the Hessian H is positive definite, and therefore admits a Cholesky decomposition H = W'*W. The following problem is solved when calling ECOS after the rewriting:
     minimize    t
     subject to  A*x <= b
           [ lb <= x <= ub ]
           [ Aeq*x == beq    ]

           ||       Wx     ||
           || (t-f'*x+1)/sqrt(2) ||_2 <= (t-f'*x-1)/sqrt(2)
If the Hessian $H$ is positive semi-definite, an eigenvalue decomposition is computed to obtain $W=H^{1/2}$. This might be slow for large matrices, but work well for diagonal Hessians. Eigenvalues which are zero are discarded from the computation of W.
\begin{equation*}
\begin{aligned}
& \underset{x, t}{\text{minimize}}
& & t \\
& \text{subject to}
& &  A x \leq b \\
& & &  l_{b} \leq x \leq u_{b} \\
& & & \begin{Vmatrix}
 W \cdot x \\ 
 \frac{t - f^{T} \cdot x + 1}{\sqrt{2}} 
\end{Vmatrix}_{2} \leq \frac{t - f^{T} \cdot x - 1}{\sqrt{2}}
\end{aligned}
\end{equation*}

In [ ]:
# define QP problem
H = np.array([[4, 1], [1, 2]])
f = np.array([1, 1])
a = np.array([[1, 1], [1, 0], [0, 1]])   # feasible solution
#a = np.matrix([[1, 1], [1, 1], [1, 1]]) # unfeasible solution
l = np.array([1, 0, 0])
u = np.array([1, 0.7, 0.7])
A = np.vstack((a, -a ))
b = np.hstack((u, -l))

In [ ]:
# construct QP
# precondition
scale = max(abs(f))
scale = 1
f     = f/scale
H     = H/scale

# number of variables
n = H.shape[1]
# H must be positive definite
W = np.linalg.cholesky(H)
# set up SOCP problem
c = np.hstack((np.zeros(n), 1)) 

# create second-order cone constraint for objective function
fhalf      = f / np.sqrt(2);
zerocolumn = np.zeros(W.shape[0])
Gquad      = np.vstack(( np.hstack((fhalf.T , -1/np.sqrt(2))),
                         np.hstack((   -W   ,  np.array([zerocolumn]).T)),
                         np.hstack((-fhalf.T,  1/np.sqrt(2)))))
hquad = np.hstack((1/np.sqrt(2), zerocolumn, 1/np.sqrt(2)))

dims = {}
G = np.vstack((np.hstack((A, np.array([np.zeros(A.shape[0])]).T)) , Gquad))  
h = np.hstack((b, hquad))
dims['l'] = A.shape[0]
dims['q'] = [W.shape[0]+2]

# sparsify
if not(isinstance(G, sparse.csc.csc_matrix)) :
    G = sparse.csc_matrix(G)

# solve
solution = ecos.solve(c,G,h,dims)
x        = solution['x'][:-1]


if solution['info']['infostring'] == 'Optimal solution found' :
    Success = True
else :
    Success = False
        
print("-----")
print("> primal solution =", x)       
print("> Status ="         , Success) 